In [ ]:
!huggingface-cli login --token 
import wandb

# Login using your API token
wandb.login(key="")
wandb.init(project="XcomT5LargeLoraOM")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `Dagoras` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Dagoras`


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: lengocquang2554 (quang123). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250921_025600-2m6knrh6
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run pious-brook-13
wandb: ⭐️ View project at https://wandb.ai/quang123/XcomT5LargeLoraOM
wandb: 🚀 View run at https://wandb.ai/quang123/XcomT5LargeLoraOM/runs/2m6knrh6


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import load_dataset
import torch
from transformers import AutoTokenizer


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

checkpoint = "google-t5/t5-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

# Preprocess

In [3]:
from datasets import DatasetDict,load_dataset


# Load the dataset
data = load_dataset("lengocquangLAB/beer-com-reviews")

# Lấy 100 dòng đầu tiên từ các split 'train' và 'test' sử dụng select()
# train_subset = data['train'].select(range(200))
# test_subset = data['test'].select(range(200))
# val_subset = data['validation'].select(range(50))

train_subset = data['train']
test_subset = data['test']
val_subset = data['validation']


# Merge the subsets back into a DatasetDict
data = DatasetDict({
    'train': train_subset,
    'test': test_subset,
    'val' : val_subset
})

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/977k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/142k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/133k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3320 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/420 [00:00<?, ? examples/s]

In [4]:
data


DatasetDict({
    train: Dataset({
        features: ['profileName', 'beerName_1', 'beerId_1', 'beerABV_1', 'beerStyle_1', 'reviewText_1', 'appearanceRate_1', 'aromaRate_1', 'palateRate_1', 'tasteRate_1', 'beerName_2', 'beerId_2', 'beerABV_2', 'beerStyle_2', 'reviewText_2', 'appearanceRate_2', 'aromaRate_2', 'palateRate_2', 'tasteRate_2', 'appearance', 'aroma', 'palate', 'taste'],
        num_rows: 3320
    })
    test: Dataset({
        features: ['profileName', 'beerName_1', 'beerId_1', 'beerABV_1', 'beerStyle_1', 'reviewText_1', 'appearanceRate_1', 'aromaRate_1', 'palateRate_1', 'tasteRate_1', 'beerName_2', 'beerId_2', 'beerABV_2', 'beerStyle_2', 'reviewText_2', 'appearanceRate_2', 'aromaRate_2', 'palateRate_2', 'tasteRate_2', 'appearance', 'aroma', 'palate', 'taste'],
        num_rows: 410
    })
    val: Dataset({
        features: ['profileName', 'beerName_1', 'beerId_1', 'beerABV_1', 'beerStyle_1', 'reviewText_1', 'appearanceRate_1', 'aromaRate_1', 'palateRate_1', 'tasteRate_1',

In [5]:
data['train']['reviewText_1'][0]
data['train']['reviewText_2'][0]

'An excellent Belgian styled ale. Pours a medium golden hue complete with very small, champagne like bubbles, and a tight foamy head. Reminscent of the strong belgian beers likes Chimay White or fellow North American brew, the Unibroue Eau Benite. Incredbly smooth tasting considering its crisp, hoppy characteristics, all of which make this a great choice for those lazy afternoons.'

In [6]:
def preprocess_function(examples):
    # Format input
    formatted_inputs = [
        f"Compare {rev1} [SEP] {rev2}"
        for rev1, rev2 in zip(examples["reviewText_1"], examples["reviewText_2"])
    ]

    # Tokenize inputs
    model_inputs = tokenizer(
        formatted_inputs,
        max_length=880,
        truncation=True,
        padding="max_length"
    )

    labels = []
    label_dicts = []

    for app, aro, pal, tas in zip(
        examples["appearance"], examples["aroma"], examples["palate"], examples["taste"]
    ):
        label_dict = {
            "appearance": app,
            "aroma": aro,
            "palate": pal,
            "taste": tas,
        }
        label_dicts.append(label_dict)

        # Chuỗi phẳng theo format yêu cầu
        label_str = (
            f"\"appearance\":{app},"
            f"\"aroma\":{aro},"
            f"\"palate\":{pal},"
            f"\"taste\":{tas}"
        )
        labels.append(label_str)

    # Tokenize labels
    labels_tokenized = tokenizer(
        text_target=labels,
        max_length=50,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels_tokenized["input_ids"]
    model_inputs["label_dict"] = label_dicts
    return model_inputs


# Map lại
tokenized_data = data.map(preprocess_function, batched=True)

sample = tokenized_data["train"][0]
print("Raw:", sample["label_dict"])
print("Label decoded:", tokenizer.decode(sample["labels"], skip_special_tokens=False))
print("Raw ids:", sample["labels"])



Map:   0%|          | 0/3320 [00:00<?, ? examples/s]

Map:   0%|          | 0/410 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Raw: {'appearance': 1.0, 'aroma': None, 'palate': None, 'taste': -1.0}
Label decoded: "appearance":1.0,"aroma":None,"palate":None,"taste":-1.0</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Raw ids: [96, 3096, 2741, 663, 121, 10, 12734, 976, 9, 3522, 9, 121, 10, 567, 782, 976, 6459, 342, 121, 10, 567, 782, 976, 17, 8323, 121, 10, 18, 12734, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
tokenized_data = tokenized_data.select_columns(['input_ids', 'attention_mask', 'labels'])

tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3320
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 410
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 420
    })
})

In [8]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=checkpoint,   # hoặc để None cũng được, nhưng để vào thì collator xử lý padding chính xác hơn
    padding=True,
    return_tensors="pt" # để HuggingFace Trainer trả tensor PyTorch
)

# Test

In [9]:
import json
from collections import defaultdict

def PRF(pred_texts, label_texts, is_test=True):
    aspects = ["appearance", "aroma", "palate", "taste"]
    positive_labels = [-1.0, 0.0, 1.0]
    negative_label = None
    default_pred = {aspect: None for aspect in aspects}

    # --- Parse ---
    def parse_flat_kv(text):
        try:
            if not text or not text.strip():
                return default_pred
            text = text.strip()
            # Tách theo dấu phẩy, sau đó split theo dấu :
            pairs = [seg.strip() for seg in text.split(",") if seg.strip()]
            result_dict = default_pred.copy()
            for pair in pairs:
                if ":" not in pair:
                    continue
                key, val = pair.split(":", 1)
                key = key.strip().strip('"')  # bỏ dấu ngoặc kép
                val = val.strip()
                if val.lower() == "null":
                    value = None
                else:
                    try:
                        value = float(val)
                    except ValueError:
                        value = None
                if key in aspects and (value in positive_labels or value is None):
                    result_dict[key] = value
            return result_dict
        except Exception:
            return default_pred

    pred = [parse_flat_kv(p) for p in pred_texts]
    label = [parse_flat_kv(l) for l in label_texts]

    # --- Helper ---
    def compute_tp_fp_fn(true_labels, pred_labels):
        tp, fp, fn = defaultdict(int), defaultdict(int), defaultdict(int)
        for true, pred in zip(true_labels, pred_labels):
            if true == pred:
                if true in positive_labels:
                    tp[true] += 1
            else:
                if true in positive_labels and pred is negative_label:
                    fn[true] += 1
                elif true is negative_label and pred in positive_labels:
                    fp[pred] += 1
                elif true in positive_labels and pred in positive_labels:
                    fn[true] += 1
                    fp[pred] += 1
        return tp, fp, fn

    def prf_from_tp_fp_fn(tp, fp, fn):
        # micro
        total_tp, total_fp, total_fn = sum(tp.values()), sum(fp.values()), sum(fn.values())
        precision_micro = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
        recall_micro = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
        f1_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0

        # macro
        precisions, recalls, f1s = [], [], []
        for cls in positive_labels:
            tpc, fpc, fnc = tp[cls], fp[cls], fn[cls]
            prec = tpc / (tpc + fpc) if (tpc + fpc) > 0 else 0
            rec = tpc / (tpc + fnc) if (tpc + fnc) > 0 else 0
            f1c = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
            precisions.append(prec)
            recalls.append(rec)
            f1s.append(f1c)
        precision_macro = sum(precisions) / len(precisions)
        recall_macro = sum(recalls) / len(recalls)
        f1_macro = sum(f1s) / len(f1s)

        return {
            "precision_macro_class": precision_macro,
            "recall_macro_class": recall_macro,
            "f1_macro_class": f1_macro,
            "precision_micro": precision_micro,
            "recall_micro": recall_micro,
            "f1_micro": f1_micro,
        }

    # --- Global (flatten all aspects) ---
    true_labels_all = [label[i][aspect] for i in range(len(label)) for aspect in aspects]
    pred_labels_all = [pred[i][aspect] for i in range(len(pred)) for aspect in aspects]
    tp_all, fp_all, fn_all = compute_tp_fp_fn(true_labels_all, pred_labels_all)
    global_scores = prf_from_tp_fp_fn(tp_all, fp_all, fn_all)

    # --- Per Aspect ---
    aspect_scores = {}
    for aspect in aspects:
        true_labels = [l[aspect] for l in label]
        pred_labels = [p[aspect] for p in pred]
        tp, fp, fn = compute_tp_fp_fn(true_labels, pred_labels)
        aspect_scores[aspect] = prf_from_tp_fp_fn(tp, fp, fn)

    # --- Macro per Aspect (trung bình các aspect) ---
    macro_precision_aspect = sum(v["precision_macro_class"] for v in aspect_scores.values()) / len(aspect_scores)
    macro_recall_aspect = sum(v["recall_macro_class"] for v in aspect_scores.values()) / len(aspect_scores)
    macro_f1_aspect = sum(v["f1_macro_class"] for v in aspect_scores.values()) / len(aspect_scores)

    # --- Return ---
    result = {
        **global_scores,
        "precision_macro_aspect": macro_precision_aspect,
        "recall_macro_aspect": macro_recall_aspect,
        "f1_macro_aspect": macro_f1_aspect,
        "per_aspect": aspect_scores
    }
    return result


In [10]:
import numpy as np

def validate_tokens(tokens):
    tokens = np.array(tokens) if not isinstance(tokens, np.ndarray) else tokens
    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
    tokens = np.where(tokens < 0, pad_token_id, tokens)  # Replace negative IDs
    return tokens


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    predictions = validate_tokens(predictions)
    labels = validate_tokens(labels)
    
    pred_texts = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    print("=== First 5 pred_texts ===")
    for i, t in enumerate(pred_texts[:5]):
        print(f"{i+1}: {t}")
    
    print("\n=== First 5 label_texts ===")
    for i, t in enumerate(label_texts[:5]):
        print(f"{i+1}: {t}")

    
    # Tính toàn bộ metrics
    metrics = PRF(pred_texts, label_texts)

    # Gom hết vào dict trả về
    result = {
        # --- Global scores ---
        "micro_precision": metrics["precision_micro"],
        "micro_recall": metrics["recall_micro"],
        "micro_f1": metrics["f1_micro"],
        "macro_precision_class": metrics["precision_macro_class"],
        "macro_recall_class": metrics["recall_macro_class"],
        "macro_f1_class": metrics["f1_macro_class"],

        # --- Macro theo aspect ---
        "macro_precision_aspect": metrics["precision_macro_aspect"],
        "macro_recall_aspect": metrics["recall_macro_aspect"],
        "macro_f1_aspect": metrics["f1_macro_aspect"],
    }

    # --- Thêm kết quả chi tiết từng aspect ---
    for aspect, scores in metrics["per_aspect"].items():
        result[f"{aspect}_precision_micro"] = scores["precision_micro"]
        result[f"{aspect}_recall_micro"] = scores["recall_micro"]
        result[f"{aspect}_f1_micro"] = scores["f1_micro"]
        result[f"{aspect}_precision_macro"] = scores["precision_macro_class"]
        result[f"{aspect}_recall_macro"] = scores["recall_macro_class"]
        result[f"{aspect}_f1_macro"] = scores["f1_macro_class"]

    return result

# Train

In [11]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import LoraConfig, get_peft_model

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.resize_token_embeddings(len(tokenizer))
# LoRA config
lora_config = LoraConfig(
    r=8,                  # rank của LoRA, nhỏ = tiết kiệm VRAM hơn
    lora_alpha=16,
    target_modules=["q", "v"],  # áp dụng LoRA vào query & value projection của attention
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)

print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Trainable params: 2359296


In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-large-lora",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps = 4,
    weight_decay=0.01,
    save_total_limit=2,               
    num_train_epochs=15,
    predict_with_generate=True,
    generation_max_length=50,
    fp16=False,
    push_to_hub=True,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_macro_f1_class",  
    greater_is_better=True,            
    report_to=["wandb"],
)


# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# Evaluate on test set for final performance
test_results = trainer.evaluate(eval_dataset=tokenized_data["test"])
print("Final Test Results:", test_results)

<ipython-input-12-316340c4a0db>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Micro Precision,Micro Recall,Micro F1,Macro Precision Class,Macro Recall Class,Macro F1 Class,Macro Precision Aspect,Macro Recall Aspect,Macro F1 Aspect,Appearance Precision Micro,Appearance Recall Micro,Appearance F1 Micro,Appearance Precision Macro,Appearance Recall Macro,Appearance F1 Macro,Aroma Precision Micro,Aroma Recall Micro,Aroma F1 Micro,Aroma Precision Macro,Aroma Recall Macro,Aroma F1 Macro,Palate Precision Micro,Palate Recall Micro,Palate F1 Micro,Palate Precision Macro,Palate Recall Macro,Palate F1 Macro,Taste Precision Micro,Taste Recall Micro,Taste F1 Micro,Taste Precision Macro,Taste Recall Macro,Taste F1 Macro
1,0.108900,0.095428,0.341501,0.207442,0.258102,0.226566,0.202631,0.213771,0.133854,0.160116,0.126673,0.334694,0.296029,0.314176,0.192600,0.252325,0.190233,0.430000,0.168627,0.242254,0.143333,0.119444,0.130303,0,0.000000,0,0.000000,0.000000,0.000000,0.318182,0.268493,0.291233,0.199481,0.268694,0.186156
2,0.091200,0.089335,0.368071,0.308837,0.335862,0.408925,0.289758,0.263745,0.330474,0.237496,0.167747,0.345912,0.397112,0.369748,0.614286,0.332356,0.184794,0.414414,0.360784,0.385744,0.138138,0.255556,0.179337,0.500000,0.005618,0.011111,0.166667,0.005376,0.010417,0.358333,0.353425,0.355862,0.402804,0.356696,0.296441
3,0.090400,0.084426,0.389691,0.351628,0.369682,0.395047,0.348941,0.368684,0.441559,0.298761,0.297270,0.357143,0.397112,0.376068,0.362679,0.371866,0.344829,0.332016,0.329412,0.330709,0.289434,0.265189,0.246766,0.538462,0.117978,0.193548,0.664474,0.116522,0.189391,0.440541,0.446575,0.443537,0.449651,0.441469,0.408095
4,0.074100,0.081640,0.422548,0.428837,0.425669,0.544804,0.408828,0.383819,0.537159,0.367670,0.328034,0.361022,0.407942,0.383051,0.516642,0.365679,0.296263,0.439560,0.470588,0.454545,0.491129,0.379297,0.319642,0.275591,0.196629,0.229508,0.567537,0.192454,0.183436,0.510582,0.528767,0.519515,0.573327,0.533251,0.512796
5,0.080800,0.073803,0.502970,0.472558,0.487290,0.522790,0.473528,0.489810,0.526361,0.432929,0.445077,0.397924,0.415162,0.406360,0.420604,0.429429,0.400360,0.512000,0.501961,0.506931,0.522569,0.460538,0.473901,0.461538,0.235955,0.312268,0.576214,0.233728,0.309528,0.586842,0.610959,0.598658,0.586057,0.608022,0.596522
6,0.072600,0.069829,0.551411,0.508837,0.529269,0.572295,0.510880,0.534455,0.562876,0.472118,0.494330,0.560284,0.570397,0.565295,0.589584,0.570715,0.561013,0.491597,0.458824,0.474645,0.511648,0.424755,0.441196,0.520000,0.292135,0.374101,0.562077,0.293385,0.381379,0.591398,0.602740,0.597015,0.588195,0.599618,0.593733
7,0.070200,0.068832,0.550096,0.531163,0.540464,0.607731,0.528547,0.535509,0.571581,0.497284,0.496891,0.616788,0.610108,0.613430,0.680541,0.600709,0.605866,0.474359,0.435294,0.453988,0.464043,0.396043,0.375580,0.458904,0.376404,0.413580,0.527889,0.378379,0.405864,0.583333,0.613699,0.598131,0.613849,0.614003,0.600253
8,0.070100,0.066258,0.568401,0.548837,0.558448,0.566719,0.561303,0.560480,0.550492,0.527493,0.530822,0.565972,0.588448,0.576991,0.567803,0.601939,0.582051,0.601770,0.533333,0.565489,0.601592,0.525315,0.559970,0.477941,0.365169,0.414013,0.477996,0.370141,0.412840,0.582474,0.619178,0.600266,0.554576,0.612576,0.568425
9,0.067600,0.063886,0.580677,0.542326,0.560847,0.593452,0.548117,0.564929,0.588102,0.523261,0.541962,0.625455,0.620939,0.623188,0.626234,0.632005,0.627645,0.547009,0.501961,0.523517,0.563079,0.490238,0.507359,0.544000,0.382022,0.448845,0.574576,0.383688,0.452759,0.581081,0.589041,0.585034,0.588518,0.587111,0.580084
10,0.055000,0.064908,0.593320,0.561860,0.577162,0.604881,0.567381,0.582112,0.594281,0.532552,0.550121,0.637324,0.653430,0.645276,0.642618,0.659033,0.648064,0.582278,0.541176,0.560976,0.609137,0.524359,0.547459,0.487395,0.325843,0.390572,0.525980,0.327872,0.400282,0.600529,0.621918,0.611036,0.599388,0.618944,0.604678


=== First 5 pred_texts ===
1: "appearance":None,"aroma":None,"palate":None,"taste":None
2: "appearance":0.0,"aroma":None,"palate":None,"taste":1.0
3: "appearance":1.0,"aroma":None,"palate":None,"taste":1.0
4: "appearance":0.0,"aroma":None,"palate":None,"taste":1.0
5: "appearance":None,"aroma":None,"palate":None,"taste":None

=== First 5 label_texts ===
1: "appearance":-1.0,"aroma":None,"palate":None,"taste":-1.0
2: "appearance":-1.0,"aroma":1.0,"palate":-1.0,"taste":0.0
3: "appearance":1.0,"aroma":None,"palate":None,"taste":1.0
4: "appearance":1.0,"aroma":None,"palate":1.0,"taste":1.0
5: "appearance":0.0,"aroma":None,"palate":None,"taste":0.0


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


=== First 5 pred_texts ===
1: "appearance":None,"aroma":None,"palate":None,"taste":-1.0
2: "appearance":0.0,"aroma":0.0,"palate":None,"taste":1.0
3: "appearance":0.0,"aroma":None,"palate":None,"taste":1.0
4: "appearance":0.0,"aroma":None,"palate":None,"taste":1.0
5: "appearance":None,"aroma":None,"palate":None,"taste":1.0

=== First 5 label_texts ===
1: "appearance":-1.0,"aroma":None,"palate":None,"taste":-1.0
2: "appearance":-1.0,"aroma":1.0,"palate":-1.0,"taste":0.0
3: "appearance":1.0,"aroma":None,"palate":None,"taste":1.0
4: "appearance":1.0,"aroma":None,"palate":1.0,"taste":1.0
5: "appearance":0.0,"aroma":None,"palate":None,"taste":0.0


...

=== First 5 pred_texts ===
1: "appearance":-1.0,"aroma":None,"palate":None,"taste":-1.0
2: "appearance":-1.0,"aroma":1.0,"palate":-1.0,"taste":0.0
3: "appearance":None,"aroma":None,"palate":None,"taste":1.0
4: "appearance":0.0,"aroma":None,"palate":None,"taste":1.0
5: "appearance":-1.0,"aroma":None,"palate":None,"taste":-1.0

=== First 5 label_texts ===
1: "appearance":-1.0,"aroma":None,"palate":None,"taste":-1.0
2: "appearance":-1.0,"aroma":1.0,"palate":-1.0,"taste":0.0
3: "appearance":1.0,"aroma":None,"palate":None,"taste":1.0
4: "appearance":1.0,"aroma":None,"palate":1.0,"taste":1.0
5: "appearance":0.0,"aroma":None,"palate":None,"taste":0.0


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


=== First 5 pred_texts ===
1: "appearance":-1.0,"aroma":None,"palate":None,"taste":-1.0
2: "appearance":-1.0,"aroma":1.0,"palate":-1.0,"taste":0.0
3: "appearance":None,"aroma":None,"palate":None,"taste":1.0
4: "appearance":0.0,"aroma":None,"palate":None,"taste":1.0
5: "appearance":-1.0,"aroma":None,"palate":None,"taste":-1.0

=== First 5 label_texts ===
1: "appearance":-1.0,"aroma":None,"palate":None,"taste":-1.0
2: "appearance":-1.0,"aroma":1.0,"palate":-1.0,"taste":0.0
3: "appearance":1.0,"aroma":None,"palate":None,"taste":1.0
4: "appearance":1.0,"aroma":None,"palate":1.0,"taste":1.0
5: "appearance":0.0,"aroma":None,"palate":None,"taste":0.0


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
No files have been modified since last commit. Skipping to prevent empty commit.


=== First 5 pred_texts ===
1: "appearance":-1.0,"aroma":-1.0,"palate":0.0,"taste":0.0
2: "appearance":-1.0,"aroma":None,"palate":None,"taste":-1.0
3: "appearance":-1.0,"aroma":-1.0,"palate":-1.0,"taste":-1.0
4: "appearance":-1.0,"aroma":-1.0,"palate":None,"taste":-1.0
5: "appearance":1.0,"aroma":None,"palate":None,"taste":-1.0

=== First 5 label_texts ===
1: "appearance":0.0,"aroma":-1.0,"palate":0.0,"taste":1.0
2: "appearance":0.0,"aroma":None,"palate":None,"taste":-1.0
3: "appearance":0.0,"aroma":-1.0,"palate":None,"taste":-1.0
4: "appearance":0.0,"aroma":-1.0,"palate":None,"taste":-1.0
5: "appearance":1.0,"aroma":None,"palate":None,"taste":0.0
Final Test Results: {'eval_loss': 0.06246815249323845, 'eval_micro_precision': 0.5587967183226983, 'eval_micro_recall': 0.5772128060263654, 'eval_micro_f1': 0.5678554886521537, 'eval_macro_precision_class': 0.5669402131159265, 'eval_macro_recall_class': 0.5801793168564727, 'eval_macro_f1_class': 0.5732791865901222, 'eval_macro_precision_aspect

In [13]:
from peft import PeftModel

# Lấy model tốt nhất
best_model = trainer.model

# Merge LoRA vào model gốc
best_model = best_model.merge_and_unload()

# Tạo thư mục mới chỉ để lưu model đã merge
merge_dir = "Xcom-after-relabeled/t5-large-lora-om"
best_model.save_pretrained(merge_dir, safe_serialization=True)
tokenizer.save_pretrained(merge_dir)

# Push thư mục này lên Hugging Face
best_model.push_to_hub("lengocquangLAB/t5-large-lora-om", repo_type="model")
tokenizer.push_to_hub("lengocquangLAB/t5-large-lora-om", repo_type="model")

README.md: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/lengocquangLAB/t5-large-lora-om/commit/b60577f586447bd2c631f18a6a64e7fe1491756f', commit_message='Upload tokenizer', commit_description='', oid='b60577f586447bd2c631f18a6a64e7fe1491756f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/lengocquangLAB/t5-large-lora-om', endpoint='https://huggingface.co', repo_type='model', repo_id='lengocquangLAB/t5-large-lora-om'), pr_revision=None, pr_num=None)

# Evaluate

In [14]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Khai báo arguments cho trainer (không cần train, chỉ để eval)
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    do_train=False,
    do_eval=True,
    logging_dir="./logs",
)

# Tạo trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # hàm bạn đã viết
    eval_dataset=tokenized_data["test"],
)

# Chạy evaluate
results = trainer.evaluate(eval_dataset=tokenized_data["test"])
print("Evaluation results:", results)


<ipython-input-14-f714e0f3cc70>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


=== First 5 pred_texts ===
1: "appearance":-1.0,"aroma":-1.0,"palate"
2: "appearance":-1.0,"aroma":None,"palate"
3: "appearance":-1.0,"aroma":-1.0,"palate"
4: "appearance":-1.0,"aroma":-1.0,"palate"
5: "appearance":1.0,"aroma":None,"palate":

=== First 5 label_texts ===
1: "appearance":0.0,"aroma":-1.0,"palate":0.0,"taste":1.0
2: "appearance":0.0,"aroma":None,"palate":None,"taste":-1.0
3: "appearance":0.0,"aroma":-1.0,"palate":None,"taste":-1.0
4: "appearance":0.0,"aroma":-1.0,"palate":None,"taste":-1.0
5: "appearance":1.0,"aroma":None,"palate":None,"taste":0.0
Evaluation results: {'eval_loss': 0.06244192644953728, 'eval_model_preparation_time': 0.0112, 'eval_micro_precision': 0.6065857885615251, 'eval_micro_recall': 0.3295668549905838, 'eval_micro_f1': 0.42708968883465526, 'eval_macro_precision_class': 0.6215970110022154, 'eval_macro_recall_class': 0.32812776238285457, 'eval_macro_f1_class': 0.42787345404736765, 'eval_macro_precision_aspect': 0.4017505598634971, 'eval_macro_recall_asp